In [1]:
import torch
from utils import tag2idx, idx2tag
from crf import Bert_BiLSTM_CRF
model = Bert_BiLSTM_CRF(tag2idx)

model.load_state_dict(torch.load('./checkpoints/finetuning/100.pt'))

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
model

Bert_BiLSTM_CRF(
  (lstm): LSTM(768, 384, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=768, out_features=16, bias=True)
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor

In [3]:
from pytorch_pretrained_bert import BertTokenizer
bert_model = '/root/workspace/qa_project/chinese_L-12_H-768_A-12'
tokenizer = BertTokenizer.from_pretrained(bert_model)

In [38]:
text = '延更丹可以与大豆异黄胴一起吃吗'
tokens = tokenizer.tokenize(text)
tokens

['延', '更', '丹', '可', '以', '与', '大', '豆', '异', '黄', '胴', '一', '起', '吃', '吗']

In [39]:
tokens = ['[CLS]'] + tokens + ['[SEP]']
tokens

['[CLS]',
 '延',
 '更',
 '丹',
 '可',
 '以',
 '与',
 '大',
 '豆',
 '异',
 '黄',
 '胴',
 '一',
 '起',
 '吃',
 '吗',
 '[SEP]']

In [40]:
xx = tokenizer.convert_tokens_to_ids(tokens)
xx

[101,
 2454,
 3291,
 710,
 1377,
 809,
 680,
 1920,
 6486,
 2460,
 7942,
 5539,
 671,
 6629,
 1391,
 1408,
 102]

In [41]:
model.eval()
xx = torch.tensor(xx).unsqueeze(0)
xx.size()

torch.Size([1, 17])

In [42]:
model = model.to(torch.device('cuda'))
xx = xx.to(torch.device('cuda'))
_, y_hat = model(torch.tensor(xx))
y_hat

tensor([[ 1, 12, 13, 13,  3,  3,  3, 12, 13, 13, 13,  3,  3,  3,  3,  3,  2]])

In [43]:
pred_tags = []
for tag in y_hat.squeeze():
    pred_tags.append(idx2tag[tag.item()])
pred_tags

['[CLS]',
 'B-DRG',
 'I-DRG',
 'I-DRG',
 'O',
 'O',
 'O',
 'B-DRG',
 'I-DRG',
 'I-DRG',
 'I-DRG',
 'O',
 'O',
 'O',
 'O',
 'O',
 '[SEP]']

In [31]:
for s, tag in zip(tokens, pred_tags):
    print(s, tag)

[CLS] [CLS]
你 O
的 O
症 O
状 O
要 O
考 O
虑 O
为 O
消 B-DSE
化 I-DSE
不 I-DSE
良 I-DSE
， O
有 O
可 O
能 O
是 O
受 O
凉 O
或 O
饮 O
食 O
不 O
当 O
引 O
起 O
的 O
， O
会 O
引 O
起 O
大 O
便 O
不 O
成 O
型 O
的 O
。 O
指 O
导 O
意 O
见 O
： O
建 O
议 O
可 O
以 O
服 O
用 O
乳 B-DRG
酸 I-DRG
菌 I-DRG
素 I-DRG
片 I-DRG
、 O
维 B-DRG
生 I-DRG
素 I-DRG
b1 I-DRG
片 O
和 O
蒙 B-DRG
脱 I-DRG
石 I-DRG
散 I-DRG
， O
注 O
意 O
休 O
息 O
， O
多 O
喝 O
开 O
水 O
， O
饮 O
食 O
以 O
清 O
淡 O
为 O
主 O
， O
禁 O
辛 O
辣 O
刺 O
激 O
性 O
食 O
物 O
和 O
油 O
腻 O
煎 O
炸 O
食 O
物 O
。 O
注 O
意 O
保 O
暖 O
， O
避 O
免 O
着 O
凉 O
。 O
[SEP] [SEP]


In [35]:
def map_ner(text, result):
    entities = []
    entity = None
    for idx , st in enumerate(result):
        if st.startswith('B'):
            entity = {}
            # entity['class'] = class_map[st.split('-')[-1]]
            entity['start'] = idx
        if entity is not None and st == 'O':
            entity['end'] = idx
            entity['name'] = text[entity['start']:entity['end']]
            entities.append(entity)
            entity = None
    return entities

In [36]:
map_ner(tokens, pred_tags)

[{'start': 9, 'end': 13, 'name': ['消', '化', '不', '良']},
 {'start': 50, 'end': 55, 'name': ['乳', '酸', '菌', '素', '片']},
 {'start': 56, 'end': 60, 'name': ['维', '生', '素', 'b1']},
 {'start': 62, 'end': 66, 'name': ['蒙', '脱', '石', '散']}]

In [1]:
from predict import main
main()

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
['罗红霉素', '头孢']
